In [2]:
import torch
from torchvision import transforms
from IPython.display import display
from utils.loss_function import SaliencyLoss

print("All imports successful")

All imports successful


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from TranSalNet_Res import TranSalNet
model = TranSalNet()
model.load_state_dict(torch.load(r'pretrained_models/TranSalNet_Res.pth', map_location=torch.device('cpu')))

model = model.to(device) 
model.eval()

print("Model imported successfully")

Model imported successfully


In [ ]:
#TARGET TWO CIRCLES
target = torch.zeros(1, 1, 288, 384)

cx1, cy1 = target.shape[2] / 1.2, target.shape[3] / 1.3
cx2, cy2 = target.shape[2] /3, target.shape[2] /4.1 

max_dist1, max_dist2 = 100, 100  # set max distances for both circles

xx, yy = torch.meshgrid(torch.linspace(0, target.shape[2] - 1, target.shape[2]), torch.linspace(0, target.shape[3] - 1, target.shape[3]))
distances1 = torch.sqrt((xx - cx1) ** 2 + (yy - cy1) ** 2)
distances2 = torch.sqrt((xx - cx2) ** 2 + (yy - cy2) ** 2)

values = torch.clamp(1 - distances1 / max_dist1, 0) + torch.clamp(1 - distances2 / max_dist2, 0)  # add the two circle values together

target[0, 0, :, :] = values
print(target.shape)

toPIL = transforms.ToPILImage()
pic = toPIL(target.squeeze())
display(pic)




In [ ]:
#TARGET CORNER

target = torch.zeros(1, 1, 288, 384)

# Set the top right of the tensor to white
target[:, :, :target.shape[2]//2, target.shape[2]//2:] = 1.0

pic = toPIL(target.squeeze())
display(pic)

In [ ]:
#INIT PATCH FOR BASIC PATCH ATTACK

def init_patch():
    #setting rand_patch to be initialized with 0's or 1's makes a significant difference in the output.
    #both outputs look very different but still acheieve the desired goal
    patch = torch.rand(1,3,288,384)

    transform = transforms.ToPILImage()
    img = transform(patch.squeeze())
    display(img)

    return patch


first_patch = init_patch()
first_patch = first_patch.type(torch.FloatTensor).to(device)

patch_saliency = model(first_patch)

toPIL = transforms.ToPILImage()
display(toPIL(patch_saliency.squeeze()))

In [ ]:
#PERFORM PATCH ATTACK ON TO TARGET SALIENCY TOWARD CERTAIN LOCATIONS
import cv2
def patch_attack(patch, target_saliency, loss_target, lr):
    #import loss functions
    loss_fn=SaliencyLoss()
    #init loss to be high as for kl div the lower the loss the better
    loss=0

    #continue updating the patch until they are similar enough, determined by a threshold set by you
    while loss_target > loss:

        print('------------------------------------------')
        patch = patch.clone().detach().requires_grad_(True)

        #get saliency map for patch
        out = model(patch)
        loss = loss_fn(out,target_saliency,loss_type='cc')
        loss.backward()

        

        #update patch values
        patch = patch + (patch.grad * lr)

        print(loss)

    return patch
    
final_patch = patch_attack(first_patch,target, loss_target = 0.8, lr=1)
display(toPIL(final_patch.squeeze()))


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(final_patch.squeeze().detach().permute(1,2,0))
plt.axis("off")